In [6]:
# Cell 1: imports & settings
import os, json, random, time
from pathlib import Path
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, datasets, models
from PIL import Image
import torch.nn.functional as F

# ---- EDIT THIS if your dataset path differs ----
DATA_DIR = r"D:\sic\dataset"   # << your dataset (contains class subfolders)
SAVE_DIR = "saved_models"
META_DIR = "meta"
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(META_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
print("Using DATA_DIR =", DATA_DIR)


Device: cpu
Using DATA_DIR = D:\sic\dataset


In [7]:
# Cell 2: dataset, transforms, and small subset (20%)
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# Use the folder that already contains class subfolders (no extra 'train' layer)
assert os.path.isdir(DATA_DIR), f"DATA_DIR not found: {DATA_DIR}"
full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
print("Classes found:", full_dataset.classes)
num_classes = len(full_dataset.classes)

# Build a 20% random subset (fast for hackathon)
n_total = len(full_dataset)
n_subset = max(50, int(n_total * 0.20))   # at least 50 samples if available
n_subset = min(n_subset, n_total)
subset_indices = random.sample(range(n_total), n_subset)
subset = Subset(full_dataset, subset_indices)

# split subset into train/val (80/20)
n_train = int(len(subset) * 0.8)
train_ds = Subset(subset, list(range(0, n_train)))
val_ds = Subset(subset, list(range(n_train, len(subset))))

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

print("Using subset size:", len(subset), "-> train:", len(train_ds), "val:", len(val_ds))
print("num_classes:", num_classes)


Classes found: ['Rice stem borer', 'green leafhopper', 'planthopper', 'rice bug', 'rice leaf roller']
Using subset size: 59 -> train: 47 val: 12
num_classes: 5


In [8]:
# Cell 3: model setup
model = models.mobilenet_v2(pretrained=True)
# Replace final classifier to match your classes
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

# Freeze backbone to speed up training (unfreeze later if you want)
for name, param in model.features.named_parameters():
    param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

print("Model ready. Trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))


C:\Users\arvin\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\arvin\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\arvin/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:03<00:00, 4.52MB/s]

Model ready. Trainable params: 6405


In [9]:
# Cell 4: quick training (3 epochs) and save artifacts
epochs = 3
start = time.time()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    epoch_loss = running_loss / max(1, len(train_loader.dataset))

    # validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            preds = out.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total if total > 0 else 0.0
    print(f"Epoch {epoch+1}/{epochs}  loss: {epoch_loss:.4f}  val_acc: {val_acc:.3f}")

# Save model weights
torch.save(model.state_dict(), os.path.join(SAVE_DIR, "pest_model.pth"))

# Save class mapping (so Streamlit will show readable names)
class_map = {str(i): name for i, name in enumerate(full_dataset.classes)}
with open(os.path.join(META_DIR, "class_indices.json"), "w", encoding="utf-8") as f:
    json.dump(class_map, f, indent=2, ensure_ascii=False)

print("Saved model to", os.path.join(SAVE_DIR, "pest_model.pth"))
print("Saved mapping to", os.path.join(META_DIR, "class_indices.json"))
print("Elapsed (s):", time.time() - start)


Epoch 1/3  loss: 1.6937  val_acc: 0.333
Epoch 2/3  loss: 1.2958  val_acc: 0.417
Epoch 3/3  loss: 1.1940  val_acc: 0.417
Saved model to saved_models\pest_model.pth
Saved mapping to meta\class_indices.json
Elapsed (s): 40.143372774124146


In [11]:
# Replacement Cell 5: automatically find a test image in DATA_DIR and run inference
import os, glob
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms, models
import torch, json

# Use the same DATA_DIR used earlier (or change here)
# Example your DATA_DIR is: D:\sic\dataset
# If you used a different DATA_DIR, change the variable below
DATA_DIR = r"D:\sic\dataset"
META_DIR = "meta"
SAVE_DIR = "saved_models"

# find any jpg/png in subdirectories (first found)
img_files = []
for ext in ("jpg","jpeg","png","JPG","PNG"):
    img_files += glob.glob(os.path.join(DATA_DIR, "**", f"*.{ext}"), recursive=True)

if not img_files:
    print(f"No images found under {DATA_DIR}. Put some images in class subfolders and try again.")
else:
    test_image_path = img_files[0]
    print("Using test image:", test_image_path)

    # load mapping
    mapping_file = os.path.join(META_DIR, "class_indices.json")
    if not os.path.isfile(mapping_file):
        print("Mapping not found:", mapping_file)
    else:
        with open(mapping_file, "r", encoding="utf-8") as f:
            class_map = json.load(f)
        inv_map = {int(k): v for k, v in class_map.items()}

        # reload model arch and weights
        num_classes = len(inv_map)
        model = models.mobilenet_v2(pretrained=False)
        model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)

        model_path = os.path.join(SAVE_DIR, "pest_model.pth")
        if not os.path.isfile(model_path):
            print("Model weights not found:", model_path)
        else:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.load_state_dict(torch.load(model_path, map_location=device))
            model = model.to(device)
            model.eval()

            # preprocess and infer
            prep = transforms.Compose([
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
            ])
            img = Image.open(test_image_path).convert("RGB")
            x = prep(img).unsqueeze(0).to(device)
            with torch.no_grad():
                out = model(x)
                probs = F.softmax(out, dim=1).cpu().numpy()[0]
            topk = probs.argsort()[-3:][::-1]
            print("Top predictions:")
            for idx in topk:
                print(f"{inv_map[idx]} — {probs[idx]:.3f}")


Using test image: D:\sic\dataset\green leafhopper\0001.jpg
Top predictions:
rice bug — 0.568
Rice stem borer — 0.263
rice leaf roller — 0.117
